In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


# hyperparameters

In [3]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = 2048
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28, 1)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
run = 10
batch_size = 256
epoch = 100

In [4]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [5]:
FGSM = onp.load('./cifar-targeted-fgsm-x.npy')
I_FGSM = onp.load('./cifar-targeted-ifgsm-x.npy')

In [6]:
y_test = onp.load("./cifar-targeted-clean-y.npy")

In [7]:
x_test = onp.load("./cifar-targeted-clean-x.npy")

In [15]:
layers = tf.keras.layers

In [16]:
img_input = layers.Input(shape=image_shape)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(img_input)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
out = layers.Dense(10)(x)

model = tf.keras.Model(inputs=img_input, outputs=out)

In [17]:
# def scheduler(epoch, lr):
#     if epoch < 5:
#         return 1e-3
#     elif epoch < 55:
#         return 1e-2
#     elif epoch < 75:
#         return 1e-3
#     else:
#         return 1e-4

In [18]:
def scheduler(epoch, lr):
    if epoch < 5:
        return 1e-3
    else:
        return 1e-2

In [19]:
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [20]:
# model.compile(optimizer=tf.keras.optimizers.SGD(),
#               loss=tf.keras.losses.MSE,
#               metrics=['accuracy'])

In [21]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [22]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 65536)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               16777472  
_________________________________________________________________
dense_1 (Dense)              (None, 10)               

In [23]:
x_train_all, x_test_all = x_train_all.reshape(-1, 32, 32, 3), x_test_all.reshape(-1, 32, 32, 3)

In [25]:
# model.fit(x_train_all, y_train_all, batch_size=batch_size, epochs=200, validation_split=0.1, callbacks=[callback])

In [26]:
model.load_weights('simple_cnn.h5')

In [27]:
# model.load_weights('./simple_cnn_mse.h5')

In [28]:
ce_loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

In [29]:
@tf.function
def targeted_fgsm(x, y_target, model, eps):
    with tf.GradientTape() as tp:
        tp.watch(x)
        y = model(x)
        loss = ce_loss(y_target, y)
    grad = tp.gradient(loss, x)
    return tf.clip_by_value(x - eps * tf.sign(grad), 0, 1)

In [30]:
@tf.function
def untargeted_fgsm(x, y_true, model, eps):
    with tf.GradientTape() as tp:
        tp.watch(x)
        y = model(x)
        loss = ce_loss(y_true, y)
    grad = tp.gradient(loss, x)
    return tf.clip_by_value(x + eps * tf.sign(grad), 0, 1)

In [31]:
def targeted_ifgsm(x, y_target, model, eps, iter_num):
    for i in range(iter_num):
        x = targeted_fgsm(x, y_target, model, eps)
    return x

In [32]:
# tmp = untargeted_fgsm(tf.convert_to_tensor(clean_x), clean_y, model, 0.03)

In [33]:
clean_x = onp.load('./cifar-clean-x.npy')
clean_y = onp.load('./cifar-clean-y.npy')

In [34]:
tmp = onp.load('./cifar-eps-time-any-npy/cifar-fgsm-eps-0.03-time-None.npy')

In [42]:
model.evaluate(clean_x, clean_y)

4/4 [==============================] - 0s 2ms/step - loss: 1.2249 - accuracy: 0.6719


[1.2249499559402466, 0.671875]

In [43]:
model.evaluate(tmp, clean_y)

4/4 [==============================] - 0s 2ms/step - loss: 2.6337 - accuracy: 0.3359


[2.6336843967437744, 0.3359375]

In [20]:
tmp = targeted_fgsm(tf.convert_to_tensor(x_test), onp.tile(onp.eye(10)[0], 4096).reshape(-1, 10), model, 0.03)

In [21]:
itmp = targeted_ifgsm(tf.convert_to_tensor(x_test), onp.tile(onp.eye(10)[0], 4096).reshape(-1, 10), model, 0.0003*1.1, 100)

In [41]:
sum(onp.argmax(model(itmp), axis=1)[onp.argmax(model(x_test), axis=1) == onp.argmax(y_test, axis=1)]==0) /\
sum(onp.argmax(model(x_test), axis=1) == onp.argmax(y_test, axis=1))

0.9648183556405354

In [24]:
onp.save('cifar-tageted-simple-cnn-ifgsm.npy' ,itmp)